<a href="https://www.kaggle.com/code/thaddeusthomas/chinook-db-sql-analytics?scriptVersionId=132794359" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Imports, connecting to SQL DB

In [1]:
!pip install ipython-sql


In [2]:
import os
import sqlite3
import plotly.express as px, plotly.graph_objects as go
import pandas as pd

In [3]:
%load_ext sql
%sql sqlite:////kaggle/input/chinook-music-store-data/chinook.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @/kaggle/input/chinook-music-store-data/chinook.db'

# Chinook analytics

## 1. Most profitable genres

The Chinook record store has just signed a deal with a new record label, and we need to select the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

* Regal	*Hip-Hop*
* Red Tone	*Punk*
* Meteor and the Girls	*Pop*
* Slim Jim Bites	*Blues*

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

This query finds out which genres sell the most tracks in the USA to make a recommendation for the three artists whose albums we should purchase for the store.

In [4]:
%%sql 

--Most popular genres in USA by number of locally sold tracks


WITH tracks_bought_in_usa 
     AS (SELECT track_id, 
                quantity, 
                i.billing_country 
         FROM   invoice_line il 
                INNER JOIN invoice i 
                        ON il.invoice_id = i.invoice_id 
         WHERE  i.billing_country = 'USA'), 
     tracks_bought_in_usa_genres 
     AS (SELECT g.NAME AS genre, 
                tracks_bought_in_usa.quantity 
         FROM   tracks_bought_in_usa 
                INNER JOIN track t 
                        ON t.track_id = tracks_bought_in_usa.track_id 
                INNER JOIN genre g 
                        ON g.genre_id = t.genre_id) 
SELECT genre, 
       Sum(quantity) AS sold_tracks, 
       Round(Cast(Sum(quantity) AS FLOAT) / (SELECT Sum(quantity) 
                                             FROM   tracks_bought_in_usa_genres) 
             * 100) 
                     AS sold_tracks_perc 
FROM   tracks_bought_in_usa_genres 
GROUP  BY genre 
ORDER  BY sold_tracks DESC 



 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


genre,sold_tracks,sold_tracks_perc
Rock,561,53.0
Alternative & Punk,130,12.0
Metal,124,12.0
R&B/Soul,53,5.0
Blues,36,3.0
Alternative,35,3.0
Pop,22,2.0
Latin,22,2.0
Hip Hop/Rap,20,2.0
Jazz,14,1.0


In [5]:
query = """
WITH tracks_bought_in_usa 
     AS (SELECT track_id, 
                quantity, 
                i.billing_country 
         FROM   invoice_line il 
                INNER JOIN invoice i 
                        ON il.invoice_id = i.invoice_id 
         WHERE  i.billing_country = 'USA'), 
     tracks_bought_in_usa_genres 
     AS (SELECT g.NAME AS genre, 
                tracks_bought_in_usa.quantity 
         FROM   tracks_bought_in_usa 
                INNER JOIN track t 
                        ON t.track_id = tracks_bought_in_usa.track_id 
                INNER JOIN genre g 
                        ON g.genre_id = t.genre_id) 
SELECT genre, 
       Sum(quantity) AS sold_tracks, 
       Round(Cast(Sum(quantity) AS FLOAT) / (SELECT Sum(quantity) 
                                             FROM   tracks_bought_in_usa_genres) 
             * 100) 
                     AS sold_tracks_perc 
FROM   tracks_bought_in_usa_genres 
GROUP  BY genre 
ORDER  BY sold_tracks DESC 
 
 """

# Make connection to database
data = sqlite3.connect('////kaggle/input/chinook-music-store-data/chinook.db')

# Convert the query output into a dataframe
genre_pop = pd.read_sql_query(query, data)
genre_pop = genre_pop.append(genre_pop.loc[9:, ['sold_tracks', 'sold_tracks_perc']].sum(), 
                            ignore_index=True)
genre_pop.iloc[-1, 0] = 'Other'
genre_pop.drop(index=list(range(9, 17)), inplace=True)

# Create a pie plot of the results
fig = px.pie(genre_pop, names='genre', values='sold_tracks', 
            title='Most Popular genres in US Market', 
            color_discrete_sequence=px.colors.sequential.RdBu_r)
fig.update_layout(legend_title_text='Genre')
fig.show()

Based on the sales of tracks across different genres in the USA, we should purchase the new album by Red Tone (Punk). It's worth keeping in mind that combined, these one genre makes up only 12% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

## 2. Sales support agents performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We need to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [6]:
%%sql

WITH total_sales 
     AS (SELECT c.support_rep_id, 
                Round(Sum(i.total)) AS total_sum 
         FROM   customer c 
                LEFT JOIN invoice i 
                       ON i.customer_id = c.customer_id 
         GROUP  BY c.support_rep_id) 
SELECT e.first_name 
       || ' ' 
       || e.last_name                                   AS name, 
       title, 
       hire_date, 
       country, 
       ts.total_sum, 
       Round(ts.total_sum / (SELECT Sum(total_sum) 
                             FROM   total_sales) * 100) AS percentage_of_sales 
FROM   employee e 
       INNER JOIN total_sales ts 
               ON ts.support_rep_id = e.employee_id 
ORDER  BY ts.total_sum DESC 



 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


name,title,hire_date,country,total_sum,percentage_of_sales
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,Canada,1732.0,37.0
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,Canada,1584.0,34.0
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,Canada,1394.0,30.0


In [7]:
query = """
WITH total_sales 
     AS (SELECT c.support_rep_id, 
                Round(Sum(i.total)) AS total_sum 
         FROM   customer c 
                LEFT JOIN invoice i 
                       ON i.customer_id = c.customer_id 
         GROUP  BY c.support_rep_id) 
SELECT e.first_name 
       || ' ' 
       || e.last_name                                   AS name, 
       title, 
       hire_date, 
       country, 
       ts.total_sum, 
       Round(ts.total_sum / (SELECT Sum(total_sum) 
                             FROM   total_sales) * 100) AS percentage_of_sales 
FROM   employee e 
       INNER JOIN total_sales ts 
               ON ts.support_rep_id = e.employee_id 
ORDER  BY ts.total_sum DESC 
 
 """

# Make connection to database
data = sqlite3.connect('////kaggle/input/chinook-music-store-data/chinook.db')

employee_sales = pd.read_sql_query(query, data)

fig = px.bar(employee_sales, x='name', y='total_sum', text='total_sum', 
            title='Total Sales Amount by Employee', 
            hover_data=['title', 'total_sum', 'hire_date'])
fig.update_xaxes(title_text='Employee')
fig.update_yaxes(title_text='Total sales value', showticklabels=False)
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
fig.update_traces(marker_color='rgb(207, 207, 207)')
fig.show()

While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates. All employees are from Canada and report to the same manager, it is hard to come to any other conclusions according to data limitations.



## 3. Country sales analytics


Query that collates data on purchases from different countries.

Where a country has only one customer, they are collected into an "Other" group, sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

Analytics
* total number of customers
* total value of sales
* average value of sales per customer
* average order value

In [8]:
%%sql

WITH total_sales_by_customer_countries 
     AS (SELECT i.customer_id, 
                i.total, 
                CASE 
                  WHEN (SELECT Count(*) 
                        FROM   customer 
                        WHERE  country = c.country) = 1 THEN "Other" 
                  ELSE c.country 
                END AS country, 
                CASE 
                  WHEN (SELECT Count(*) 
                        FROM   customer 
                        WHERE  country = c.country) = 1 THEN 1 
                  ELSE 0 
                END AS only_one_customer 
         FROM   invoice i 
                INNER JOIN customer c 
                        ON i.customer_id = c.customer_id), 
    
    
     total_sales_by_customer_countries_analytincs 
     AS (SELECT only_one_customer, 
                country, 
                ( Count(DISTINCT customer_id) )                        AS 
                   unique_customers, 
                Round(Sum(total))                                      AS 
                   total_value_sales, 
                Round(Sum(total) / ( Count(DISTINCT customer_id) ), 2) AS 
                   avg_lifetime_sales, 
                Round(Sum(total) / ( Count(customer_id) ), 2)          AS 
                avg_order 
         FROM   total_sales_by_customer_countries 
         GROUP  BY country) 
    
    
SELECT country, 
       unique_customers, 
       total_value_sales, 
       avg_lifetime_sales, 
       avg_order 
FROM   total_sales_by_customer_countries_analytincs 
ORDER BY only_one_customer, total_value_sales DESC

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


country,unique_customers,total_value_sales,avg_lifetime_sales,avg_order
USA,13,1040.0,80.04,7.94
Canada,8,536.0,66.95,7.05
Brazil,5,428.0,85.54,7.01
France,5,389.0,77.81,7.78
Germany,4,335.0,83.66,8.16
Czech Republic,2,273.0,136.62,9.11
United Kingdom,3,246.0,81.84,8.77
Portugal,2,185.0,92.57,6.38
India,2,183.0,91.58,8.72
Other,15,1095.0,73.0,7.45


In [9]:
query = '''

WITH total_sales_by_customer_countries 
     AS (SELECT i.customer_id, 
                i.total, 
                CASE 
                  WHEN (SELECT Count(*) 
                        FROM   customer 
                        WHERE  country = c.country) = 1 THEN "Other" 
                  ELSE c.country 
                END AS country, 
                CASE 
                  WHEN (SELECT Count(*) 
                        FROM   customer 
                        WHERE  country = c.country) = 1 THEN 1 
                  ELSE 0 
                END AS only_one_customer 
         FROM   invoice i 
                INNER JOIN customer c 
                        ON i.customer_id = c.customer_id), 
    
    
     total_sales_by_customer_countries_analytincs 
     AS (SELECT only_one_customer, 
                country, 
                ( Count(DISTINCT customer_id) )                        AS 
                   unique_customers, 
                Round(Sum(total))                                      AS 
                   total_value_sales, 
                Round(Sum(total) / ( Count(DISTINCT customer_id) ), 2) AS 
                   avg_lifetime_sales, 
                Round(Sum(total) / ( Count(customer_id) ), 2)          AS 
                avg_order 
         FROM   total_sales_by_customer_countries 
         GROUP  BY country) 
    
    
SELECT country, 
       unique_customers, 
       total_value_sales, 
       avg_lifetime_sales, 
       avg_order 
FROM   total_sales_by_customer_countries_analytincs 
ORDER BY only_one_customer, total_value_sales DESC
'''


country_data = pd.read_sql_query(query, data)

# Create bar plot of data
fig = px.bar(
    country_data,
    x='total_value_sales',
    y='country',
    title='Sales by Country',
    hover_data=country_data.columns[1:], 
    text='total_value_sales', 
    orientation='h'
)
fig.update_xaxes(tickangle=90, showticklabels=False)
fig.update_traces(marker_color='rgb(207, 207, 207)')
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [10]:
fig = px.scatter_matrix(country_data.iloc[:, 1:])
fig.update_layout(height=700, width=970, title='Country Statistics Correlations', title_x=.5)
fig.update_traces(showupperhalf=False, diagonal_visible=False)
fig.show()

Based on the data, there may be opportunity in the following countries:
* Czech Republic
* United Kingdom
* India

## 4. Full album purchases vs. single tracks stat

The Chinook store allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [11]:
%%sql



WITH invoice_track_album 
     AS (SELECT il.invoice_id, 
                il.track_id, 
                t.album_id 
         FROM   invoice_line il 
                INNER JOIN track t 
                        ON t.track_id = il.track_id), 
    
     invoice_album_categorization 
     AS (SELECT ita.invoice_id, 
                ita.album_id, 
                CASE 
                  WHEN (SELECT t.track_id 
                        FROM   track t 
                        WHERE  t.album_id = ita.album_id 
                        EXCEPT 
                        SELECT subita.track_id 
                        FROM   invoice_track_album subita 
                        WHERE  subita.invoice_id = ita.invoice_id) IS NULL 
                       AND (SELECT subita.track_id 
                            FROM   invoice_track_album subita 
                            WHERE  subita.invoice_id = ita.invoice_id 
                            EXCEPT 
                            SELECT t.track_id 
                            FROM   track t 
                            WHERE  t.album_id = ita.album_id) IS NULL THEN 
                  'full' 
                  ELSE 'partial' 
                END AS full_album 
         FROM   invoice_track_album ita 
         GROUP  BY invoice_id) 
    
SELECT full_album, 
       Count(*) 
       AS number_sold, 
       Round(Cast(Count(*) AS FLOAT) / (SELECT Count(*) 
                                        FROM   invoice_album_categorization) * 
             100) AS 
       percentage_sold 
FROM   invoice_album_categorization 
GROUP  BY full_album 

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


full_album,number_sold,percentage_sold
full,114,19.0
partial,500,81.0


In [12]:
query = '''

WITH invoice_track_album 
     AS (SELECT il.invoice_id, 
                il.track_id, 
                t.album_id 
         FROM   invoice_line il 
                INNER JOIN track t 
                        ON t.track_id = il.track_id), 
    
     invoice_album_categorization 
     AS (SELECT ita.invoice_id, 
                ita.album_id, 
                CASE 
                  WHEN (SELECT t.track_id 
                        FROM   track t 
                        WHERE  t.album_id = ita.album_id 
                        EXCEPT 
                        SELECT subita.track_id 
                        FROM   invoice_track_album subita 
                        WHERE  subita.invoice_id = ita.invoice_id) IS NULL 
                       AND (SELECT subita.track_id 
                            FROM   invoice_track_album subita 
                            WHERE  subita.invoice_id = ita.invoice_id 
                            EXCEPT 
                            SELECT t.track_id 
                            FROM   track t 
                            WHERE  t.album_id = ita.album_id) IS NULL THEN 
                  'full' 
                  ELSE 'partial' 
                END AS full_album 
         FROM   invoice_track_album ita 
         GROUP  BY invoice_id) 
    
SELECT full_album, 
       Count(*) 
       AS number_sold, 
       Round(Cast(Count(*) AS FLOAT) / (SELECT Count(*) 
                                        FROM   invoice_album_categorization) * 
             100) AS 
       percentage_sold 
FROM   invoice_album_categorization 
GROUP  BY full_album 
'''


# Store query output in DataFrame
album_or_not = pd.read_sql_query(query, data)

# Create bar plot of DataFrame
fig = px.bar(album_or_not, x='full_album', y='number_sold', 
            title='Number of Album vs Single Purchases', 
            text='number_sold', 
            hover_data=['full_album', 'percentage_sold', 'number_sold'])
fig.update_traces(marker_color='rgb(207, 207, 207)')
fig.update_yaxes(showticklabels=False)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.show()

Album purchases account for 18.6% of purchases. Based on this data, it's possible to recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.


## 5. Most popular artists in playlists

In [13]:
%%sql

WITH playlist_track_artist AS 
( 
          SELECT    p.NAME    playlist, 
                    t.NAME    track, 
                    alb.title album, 
                    art.NAME  artist 
          FROM      playlist p 
          LEFT JOIN playlist_track 
          ON        playlist_track.playlist_id = p.playlist_id 
          LEFT JOIN track t 
          ON        t.track_id = playlist_track.track_id 
          LEFT JOIN album alb 
          ON        alb.album_id = t.album_id 
          LEFT JOIN artist art 
          ON        art.artist_id = alb.artist_id) 
SELECT   artist, 
         Count(*) in_playlists 
FROM     playlist_track_artist 
GROUP BY artist 
ORDER BY in_playlists DESC limit 10



 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


artist,in_playlists
Iron Maiden,516
U2,333
Metallica,296
Led Zeppelin,252
Deep Purple,226
Lost,184
Pearl Jam,177
Faith No More,145
Eric Clapton,145
Lenny Kravitz,143


In [14]:
query = '''

WITH playlist_track_artist AS 
( 
          SELECT    p.NAME    playlist, 
                    t.NAME    track, 
                    alb.title album, 
                    art.NAME  artist 
          FROM      playlist p 
          LEFT JOIN playlist_track 
          ON        playlist_track.playlist_id = p.playlist_id 
          LEFT JOIN track t 
          ON        t.track_id = playlist_track.track_id 
          LEFT JOIN album alb 
          ON        alb.album_id = t.album_id 
          LEFT JOIN artist art 
          ON        art.artist_id = alb.artist_id) 
SELECT   artist, 
         Count(*) in_playlists 
FROM     playlist_track_artist 
GROUP BY artist 
ORDER BY in_playlists DESC limit 10


'''

# Store the output of query into a DataFrame
tracks_sold = pd.read_sql_query(query, data)

# Create bar plot of DataFrame
fig = px.bar(tracks_sold, x='in_playlists', y='artist', orientation='h', text='in_playlists', 
            title='Top 10 Number of Tracks Sold by Artist')
fig.update_xaxes(showticklabels=False)
fig.update_traces(marker_color='rgb(207, 207, 207)')
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.show()

The list contains most popular artists used throughout playlists in Chinook DB.

## 6. Number of tracks purchased at least once

In [15]:
%%sql

SELECT Count(DISTINCT track_id) n_purchased, 
       (SELECT Count (*) 
        FROM   track)           n_total, 
       Round (Cast(Count(DISTINCT track_id) AS FLOAT) / (SELECT Count (*) 
                                                         FROM   track) * 100) 
                                percentage 
FROM   invoice_line 

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


n_purchased,n_total,percentage
1806,3503,52.0


# SQL experiments

In [16]:

%%sql

-- invoice information by invoice ID



SELECT
    il.track_id,
    t.name track_name,
    art.name artist_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
INNER JOIN album a ON a.album_id = t.album_id
INNER JOIN artist art on a.artist_id = art.artist_id
WHERE il.invoice_id = 4;

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


track_id,track_name,artist_name,track_type,unit_price,quantity
3448,"Lamentations of Jeremiah, First Set \ Incipit Lamentatio",The King's Singers,Protected AAC audio file,0.99,1
2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
1872,Attitude,Metallica,MPEG audio file,0.99,1
748,Dealer,Deep Purple,MPEG audio file,0.99,1
1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


In [17]:
%%sql

--From which albums the most songs have been purchased.

SELECT ta.album_title AS album, 
       ta.artist_name AS artist, 
       Count(*)       AS tracks_purchased 
FROM   invoice_line il 
       INNER JOIN (SELECT track.track_id, 
                          alb.title album_title, 
                          art.name  artist_name 
                   FROM   track 
                          INNER JOIN album alb 
                                  ON alb.album_id = track.album_id 
                          INNER JOIN artist art 
                                  ON art.artist_id = alb.artist_id) ta 
               ON ta.track_id = il.track_id 
GROUP  BY 1 
ORDER  BY 3 DESC 
LIMIT  5 

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


## Recursive joins

In [18]:
%%sql

--Recursive join report about employees and their supervisors.

SELECT e1.first_name 
       || "" 
       || e1.last_name AS employee_name, 
       e1.title        employee_title, 
       e2.first_name 
       || "" 
       || e2.last_name AS supervisor_name, 
       e2.title        supervisor_title 
FROM   employee e1 
       LEFT JOIN employee e2 
              ON e1.reports_to = e2.employee_id 
ORDER  BY 1 

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


employee_name,employee_title,supervisor_name,supervisor_title
AndrewAdams,General Manager,None,None
JanePeacock,Sales Support Agent,NancyEdwards,Sales Manager
LauraCallahan,IT Staff,MichaelMitchell,IT Manager
MargaretPark,Sales Support Agent,NancyEdwards,Sales Manager
MichaelMitchell,IT Manager,AndrewAdams,General Manager
NancyEdwards,Sales Manager,AndrewAdams,General Manager
RobertKing,IT Staff,MichaelMitchell,IT Manager
SteveJohnson,Sales Support Agent,NancyEdwards,Sales Manager


## Join on subquery

In [19]:
%%sql

--Summarizing purchases of each customer

SELECT cust.first_name 
       || ' ' 
       || cust.last_name           customer_name, 
       invcust.number_of_purchases number_of_purchases, 
       invcust.total_spent         total_spent, 
       invcust.customer_category   customer_category 
FROM   customer cust 
       INNER JOIN (SELECT customer_id, 
                          Count(*)   AS number_of_purchases, 
                          Sum(total) AS total_spent, 
                          CASE 
                            WHEN Sum(total) < 40 THEN 'small spender' 
                            WHEN Sum(total) <= 100 THEN 'regular' 
                            ELSE 'big spender' 
                          END        AS customer_category 
                   FROM   invoice 
                   GROUP  BY customer_id) invcust 
               ON invcust.customer_id = cust.customer_id 
ORDER  BY customer_name 

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


customer_name,number_of_purchases,total_spent,customer_category
Aaron Mitchell,8,70.28999999999999,regular
Alexandre Rocha,10,69.3,regular
Astrid Gruber,9,69.3,regular
Bjørn Hansen,9,72.27000000000001,regular
Camille Bernard,9,79.2,regular
Daan Peeters,7,60.38999999999999,regular
Dan Miller,12,95.03999999999999,regular
Diego Gutiérrez,5,39.6,small spender
Dominique Lefebvre,9,72.27,regular
Eduardo Martins,12,60.39,regular


## WITH as subquery

In [20]:
%%sql

-- A query that shows summary data for every playlist in the Chinook database

-- LEFT JOINs between the tables inside named subquery to include all playlists whether they have tracks added to them or not.

WITH playlist_extended 
     AS (SELECT playlist.playlist_id, 
                playlist.NAME             playlist_name, 
                track.NAME                track_name, 
                track.milliseconds / 1000 AS length 
         FROM   playlist 
                LEFT JOIN playlist_track 
                       ON playlist.playlist_id = playlist_track.playlist_id 
                LEFT JOIN track 
                       ON playlist_track.track_id = track.track_id) 
SELECT playlist_id, 
       playlist_name, 
       Count(track_name)    AS number_of_tracks, 
       Sum(length) AS length_seconds 
FROM   playlist_extended 
GROUP  BY playlist_id 
ORDER  BY playlist_id ASC 

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


playlist_id,playlist_name,number_of_tracks,length_seconds
1,Music,3290,876049
2,Movies,0,None
3,TV Shows,213,500987
4,Audiobooks,0,None
5,90’s Music,1477,397970
6,Audiobooks,0,None
7,Movies,0,None
8,Music,3290,876049
9,Music Videos,1,294
10,TV Shows,213,500987


## Creating a view

In [21]:
%%sql

-- view of customers that have purchased more than 90 from the store.

DROP VIEW IF EXISTS customer_gt_90_dollars;
CREATE TEMPORARY VIEW customer_gt_90_dollars AS 
SELECT     customer.* 
FROM       customer 
INNER JOIN 
           ( 
                SELECT   Sum(total) total_sum, 
                         customer_id 
                FROM     invoice 
                GROUP BY customer_id 
                HAVING   total_sum > 90
           ) ts ON customer.customer_id = ts.customer_id;


SELECT * 
FROM   customer_gt_90_dollars
LIMIT  10;



 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.
Done.
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3


In [22]:
%%sql

-- View of all customers that live in the USA.

DROP VIEW IF EXISTS customer_usa;
CREATE TEMPORARY VIEW customer_usa AS 
SELECT * 
FROM   customer 
WHERE  country = "USA";

SELECT * 
FROM   customer_usa
LIMIT  10;

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.
Done.
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
23,John,Gordon,None,69 Salem Street,Boston,MA,USA,2113,+1 (617) 522-1333,None,johngordon22@yahoo.com,4
24,Frank,Ralston,None,162 E Superior Street,Chicago,IL,USA,60611,+1 (312) 332-3232,None,fralston@gmail.com,3
25,Victor,Stevens,None,319 N. Frances Street,Madison,WI,USA,53703,+1 (608) 257-0597,None,vstevens@yahoo.com,5


## UNION both views

In [23]:
%%sql

SELECT * 
FROM   customer_usa 
UNION 
SELECT * 
FROM   customer_gt_90_dollars 

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4


## JOIN on INTERSECT with groupping

In [24]:
%%sql

-- a query that works out how many customers that are in the USA and 
-- have purchased more than $90 are assigned to each sales support agent.

SELECT first_name 
       || ' ' 
       || last_name              AS employee_name, 
       Count(usa_90.customer_id) AS customers_usa_gt_90 
FROM   employee e 
       LEFT JOIN (SELECT customer_id, 
                         support_rep_id 
                  FROM   customer_usa 
                  INTERSECT 
                  SELECT customer_id, 
                         support_rep_id 
                  FROM   customer_gt_90_dollars) usa_90 
              ON usa_90.support_rep_id = e.employee_id 
WHERE  title = 'Sales Support Agent' 
GROUP  BY usa_90.support_rep_id 
ORDER  BY employee_name 

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


employee_name,customers_usa_gt_90
Jane Peacock,0
Margaret Park,2
Steve Johnson,2


## Multiple named WITH clauses and JOIN

In [25]:
%%sql

--The first named subquery returns all customers that are from India.
--The second named subquery calculates the sum total for every customer.
--The main query joins the two named subqueries


WITH indian_customers 
     AS (SELECT * 
         FROM   customer 
         WHERE  country = 'India'), 
     total_sum 
     AS (SELECT customer_id, 
                Sum(total) AS total_purchases 
         FROM   invoice 
         GROUP  BY customer_id) 
SELECT first_name 
       || ' ' 
       || last_name              AS customer_name, 
       total_sum.total_purchases AS total_purchases 
FROM   indian_customers 
       INNER JOIN total_sum 
               ON total_sum.customer_id = indian_customers.customer_id 
ORDER  BY customer_name 

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


customer_name,total_purchases
Manoj Pareek,111.86999999999999
Puja Srivastava,71.28


## WITH summary statistics

In [26]:
%%sql

WITH total_sum_by_customer 
     AS (SELECT c.customer_id  AS customer_id, 
                c.first_name 
                || ' ' 
                || c.last_name AS customer_name, 
                Sum(i.total)   AS total_purchased, 
                c.country      AS country 
         FROM   invoice i 
                INNER JOIN customer c 
                        ON c.customer_id = i.customer_id 
         GROUP  BY i.customer_id) 
SELECT country, 
       customer_name, 
       Max(total_purchased) AS total_purchased 
FROM   total_sum_by_customer 
GROUP  BY country 
ORDER  BY country 

 * sqlite:////kaggle/input/chinook-music-store-data/chinook.db
Done.


country,customer_name,total_purchased
Argentina,Diego Gutiérrez,39.6
Australia,Mark Taylor,81.18
Austria,Astrid Gruber,69.3
Belgium,Daan Peeters,60.38999999999999
Brazil,Luís Gonçalves,108.89999999999998
Canada,François Tremblay,99.99
Chile,Luis Rojas,97.02000000000001
Czech Republic,František Wichterlová,144.54000000000002
Denmark,Kara Nielsen,37.61999999999999
Finland,Terhi Hämäläinen,79.2
